<a href="https://colab.research.google.com/github/sidereomundi/RedSequence/blob/main/red_sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This notebook demonstrates how to identify and model the galaxy red sequence using COSMOS, DES and SPT data. Explanatory text is added throughout to clarify each step.

### Installing dependencies

The first code cell installs the Python packages required for the analysis, including `emcee` for MCMC sampling and `pygtc` for plotting utilities.

In [ ]:
%pip install emcee pygtc
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt
import emcee
import pygtc
from tqdm import tqdm
from scipy.interpolate import interp1d
from google.colab import drive
from astropy.stats import sigma_clip
drive.mount('/content/drive')

### Loading COSMOS data

The COSMOS catalog is read from a FITS file stored on Google Drive. We inspect a few rows to ensure the data are loaded correctly.

In [ ]:
cosmos = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/cosmosM.fit")
cosmos[0:3]

### Computing rest-frame magnitudes and UVJ selection

COSMOS photometry is converted to U, V and J magnitudes. We apply a colour selection using the helper function `UVJsel` to flag passive galaxies.

In [ ]:
U = -2.5*np.log10(cosmos["EZrestU"]) + 23.9
V = -2.5*np.log10(cosmos["EZrestV"]) + 23.9
J = -2.5*np.log10(cosmos["EZrestJ"]) + 23.9
cosmos["U"] = U
cosmos["V"] = V
cosmos["J"] = J
z = cosmos["EZzphot"]
cut = (cosmos["EZzphot"]<1.5) & (cosmos["SolModel"] != "PointSource") & (cosmos["FModel"] == 0) & (cosmos["loglpMassmed"] > 8.5)
#cut = np.isfinite(U)

def UVJsel(cosmos):
    passive = np.zeros(len(cosmos),dtype=bool)
    U = -2.5*np.log10(cosmos["EZrestU"]) + 23.9
    V = -2.5*np.log10(cosmos["EZrestV"]) + 23.9
    J = -2.5*np.log10(cosmos["EZrestJ"]) + 23.9
    z = cosmos["EZzphot"]
    passive[((U -V) > (0.88*(V - J) + 0.69)) & (0.0 < z) & (z < 0.5) & (U-V>1.3) & (V-J<1.6)] = True
    passive[((U -V) > 0.88*(V - J) + 0.59) & (0.5 <= z) & (z < 1.0) & (U-V>1.3) & (V-J<1.6)] = True
    passive[((U -V) > 0.88*(V - J) + 0.49) & (1.0 <= z) & (z < 2.0) & (U-V>1.3) & (V-J<1.6)] = True
    cosmos["passive"] = passive
#    return(passive)
UVJsel(cosmos)

### Visualising the UVJ diagram

A 2D histogram illustrates how galaxies populate the UVJ colour space. Selection boundaries for different redshift ranges are overplotted.

In [ ]:
def plotz05():
    x = np.linspace(-5,5,1000)
    y = np.zeros_like(x)
    y = (0.88*x + 0.69)
    y[y<1.3] = 1.3
    x[x>1.6] = 1.6
    return x,y

def plotz10():
    x = np.linspace(-5,5,1000)
    y = np.zeros_like(x)
    y = (0.88*x + 0.59)
    y[y<1.3] = 1.3
    x[x>1.6] = 1.6
    return x,y

def plotz20():
    x = np.linspace(-5,5,1000)
    y = np.zeros_like(x)
    y = (0.88*x + 0.49)
    y[y<1.3] = 1.3
    x[x>1.6] = 1.6
    return x,y

In [ ]:
UV = U[cut]-V[cut]
VJ = V[cut]-J[cut]
#plt.plot(VJ,UV,'.',alpha=0.1)
#plt.plot(cosmos["V"][cosmos["passive"]]-cosmos["J"][cosmos["passive"]], cosmos["U"][cosmos["passive"]]-cosmos["V"][cosmos["passive"]],'.',alpha=0.01)
#plt.ylim([0.8,2.2])

pcut = np.array(cosmos["passive"][cut])
plt.figure(figsize=(8, 6)) # Set the figure size, optional
plt.hist2d(VJ, UV, bins=50, cmap='ocean_r',range=[(0,2.3),(0,2.5)]) # Adjust the number of bins and colormap as needed
plt.hist2d(VJ[pcut], UV[pcut], bins=50, cmap='hot_r',range=[(0,2.3),(0,2.5)],alpha=0.45) # Adjust the number of bins and colormap as needed
#plt.plot(cosmos["V"][~cosmos["passive"]]-cosmos["J"][~cosmos["passive"]], cosmos["U"][~cosmos["passive"]]-cosmos["V"][~cosmos["passive"]],'g.',alpha=0.005)
#plt.plot(cosmos["V"][cosmos["passive"]]-cosmos["J"][cosmos["passive"]], cosmos["U"][cosmos["passive"]]-cosmos["V"][cosmos["passive"]],'r.',alpha=0.01)
#plt.colorbar() # Show the color bar representing the counts

x05,y05 = plotz05()
plt.plot(x05,y05,'k:',label='z<0.5')
x10,y10 = plotz10()
plt.plot(x10,y10,'k--',label='0.5<z<1')
x20,y20 = plotz20()
plt.plot(x20,y20,'k',label='1<z<2')

plt.ylabel('UV') # Set x-axis label
plt.xlabel('VJ') # Set y-axis label
plt.title('2D Histogram of UV vs VJ') # Set title

plt.legend()

### Filtering passive galaxies

After selecting passive objects we restrict the catalogue to only these galaxies for subsequent analysis.

In [ ]:
cosmos = cosmos[cut]

### DES filter definitions

Arrays containing the effective wavelengths of DES filters are defined. These are later used in the colour--magnitude relation model.

In [ ]:
filters = np.array([473, 642, 784, 926])
filters_b = np.array([398, 568, 710, 850])
filters_r = np.array([548, 716, 857, 1002])
print(filters_b/400-1)
print(filters_r/400-1)

In [ ]:
MLAB = ["MAG_G","MAG_R","MAG_I","MAG_Z"]
MLABERR = ["MAGERR_G","MAGERR_R","MAGERR_I","MAGERR_Z"]

### Loading DES photometry

DES data are read from disk and column names are standardised. Magnitude limits are applied to ensure quality photometry.

In [ ]:
des = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/DEScosmosy3deep.fits")
des["MAG_G"] = des["BDF_MAG_G"]
des["MAG_R"] = des["BDF_MAG_R"]
des["MAG_I"] = des["BDF_MAG_I"]
des["MAG_Z"] = des["BDF_MAG_Z"]

des["MAGERR_G"] = des["BDF_MAG_ERR_DERED_G"]
des["MAGERR_R"] = des["BDF_MAG_ERR_DERED_R"]
des["MAGERR_I"] = des["BDF_MAG_ERR_DERED_I"]
des["MAGERR_Z"] = des["BDF_MAG_ERR_DERED_Z"]

mlim = [26.46,25.73,25.54,24.97]
blab = ["G","R","I","Z"]

for j,i in enumerate(blab):
    des = des[des["MAG_"+i]<mlim[j]]

for ilab in range(3):
    test = (
    (des[MLAB[ilab]] > 15) &
    (des[MLAB[ilab + 1]] > 15) &
    (des[MLABERR[ilab]] > 0) &
    (des[MLABERR[ilab + 1]] > 0)
    )
    des = des[test]

### Cross-matching COSMOS with DES

Using `astropy` we match sources between COSMOS and DES within one arcsecond to obtain consistent colours.

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
c1 = SkyCoord(cosmos["RAJ2000"], cosmos["DEJ2000"])
c2 = SkyCoord(des["RA"]*u.deg, des["DEC"]*u.deg)

In [ ]:
max_sep = 1.0 * u.arcsec
idx, d2d, d3d = c1.match_to_catalog_sky(c2)
sep_constraint = d2d < max_sep
cosmosm = cosmos[sep_constraint]
desm = des[idx[sep_constraint]]

In [ ]:
cosmosm[0:4], len(cosmosm)

In [ ]:
desm[0:4]

### Colour--magnitude diagnostic plots

These cells visualise galaxy colours versus magnitude and use sigma clipping to identify outliers.

In [ ]:
test = (cosmosm["EZzphot"]<0.5) & (cosmosm["EZzphot"]>0.45) & (cosmosm["passive"] )

In [ ]:
plt.plot(desm["MAG_I"][test],desm["MAG_R"][test]-desm["MAG_I"][test],'.',alpha=0.3)
sigmacl = sigma_clip(desm["MAG_R"][test]-desm["MAG_I"][test],sigma=3., cenfunc='median',stdfunc='mad_std',maxiters=None, masked=True)
sigmaclipped = sigmacl.mask
plt.plot(desm["MAG_I"][test][sigmaclipped],desm["MAG_R"][test][sigmaclipped]-desm["MAG_I"][test][sigmaclipped],'.',alpha=0.3)
#plt.ylim(-1,3)

In [ ]:
plt.plot(des["MAG_I"],des["MAG_R"]-des["MAG_I"],'.',alpha=0.3)
plt.ylim(-1,3)
plt.xlim(15,25)

In [ ]:

zrange = np.arange(0.05,1.5,0.075)
dz = zrange[1]-zrange[0]

for i,j in enumerate(zrange):
    test = (cosmosm["EZzphot"]<j+dz) & (cosmosm["EZzphot"]>=j) & (cosmosm["passive"])
    plt.plot(desm["MAG_I"][test][::10],desm["MAG_R"][test][::10]-desm["MAG_I"][test][::10],'.',alpha=1,label=str(j))
plt.legend()
plt.xlim(15,25)
plt.ylim(-1,2)

### Fitting the colour--magnitude relation

In discrete redshift bins we fit a linear relation to the colours via likelihood maximisation. The resulting parameters are interpolated as smooth functions of redshift.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
from scipy.stats import norm
from scipy.optimize import minimize

# Funzioni già definite
def CMRL(theta, x, y, xerr, yerr):
    A, B, s,p,m0,ds = theta
    ymod = A + B * (x-np.median(x))
    ymoderr2 = yerr**2 + s**2 + (xerr * B)**2
    return np.sum(np.log(p*norm.pdf(y, loc=ymod, scale=np.sqrt(ymoderr2))+(1-p)*norm.pdf(y, loc=m0, scale=s+ds)))

def CMRL_clipped(theta, x, y, xerr, yerr):
    A, B, s = theta
    ymod = A + B * (x-np.median(x))
    ymoderr2 = yerr**2 + s**2 + (xerr * B)**2
    return norm.logpdf(y, loc=ymod, scale=np.sqrt(ymoderr2)).sum()

def prior(theta):
    A, B, s,p,m0,ds = theta
    return 0 if (s > 0) & (ds>0) & (p>0) & (p<1) else -np.inf

def prior_clipped(theta):
    A, B, s = theta
    return 0 if (s > 0)  else -np.inf

def CMRp(theta, x, y, xerr, yerr):
    p = prior(theta)
    if p == 0:
        return CMRL(theta, x, y, xerr, yerr) + p
    else:
        return -np.inf

def CMRp_clipped(theta, x, y, xerr, yerr):
    p = prior_clipped(theta)
    if p == 0:
        return CMRL_clipped(theta, x, y, xerr, yerr) + p
    else:
        return -np.inf

In [ ]:
results = np.zeros((len(zrange), 4, 3))  # 3 per ilab, 3 per A, B, s

# Loop su ilab
for ilab in range(3):
    for i, j in enumerate(zrange):


        test1 = np.where((cosmosm["EZzphot"] < (j + dz)) & (cosmosm["EZzphot"] >= j) & (cosmosm["passive"]))[0]
        sigmacl = sigma_clip(desm[MLAB[ilab]][test1]-desm[MLAB[ilab+1]][test1],sigma=3., cenfunc='median',stdfunc='mad_std',maxiters=None, masked=True)
        sigmaclipped = sigmacl.mask
        test = test1[sigmaclipped]
        x = desm[MLAB[ilab + 1]][test]
        y = desm[MLAB[ilab]][test] - desm[MLAB[ilab + 1]][test]
        xerr = desm[MLABERR[ilab + 1]][test]
        yerr = np.sqrt(desm[MLABERR[ilab]][test]**2 + desm[MLABERR[ilab + 1]][test]**2)

        if i==0:
            p0 = np.random.rand(6)
            p0[2] = np.abs(p0[2])  # Assicurati che s sia positivo
            p0[5] = np.abs(p0[5])  # Assicurati che ds sia positivo
        else:
            p0=soln.x
        np.random.seed(42)
        nll = lambda *args: -CMRp(*args)
        initial = p0
        soln = minimize(nll, initial, args=(x, y, xerr, yerr),method="Powell")
        results[i, 0:3, ilab] = soln.x[0:3]
        results[i, 3, ilab] = np.median(x)

# Creazione delle funzioni di interpolazione per ogni parametro e ogni ilab
interp_funcs = {}
for idx, param in enumerate(['A', 'B', 's','med']):
    interp_funcs[param] = [interp1d(zrange, results[:, idx, k], kind='quadratic', fill_value="extrapolate") for k in range(3)]

def CMR(z):
    """ Restituisce i valori interpolati di A, B, s, e med per ogni ilab dato un z. """
    return [func(z) for sublist in interp_funcs.values() for func in sublist]



In [ ]:
results = np.zeros((len(zrange), 4, 3))  # 3 per ilab, 3 per A, B, s

# Loop su ilab
for ilab in range(3):
    for i, j in enumerate(zrange):


        test = (cosmosm["EZzphot"] < (j + dz)) & (cosmosm["EZzphot"] >= j) & (cosmosm["passive"])
        x = desm[MLAB[ilab + 1]][test]
        y = desm[MLAB[ilab]][test] - desm[MLAB[ilab + 1]][test]

        xerr = desm[MLABERR[ilab + 1]][test]
        yerr = np.sqrt(desm[MLABERR[ilab]][test]**2 + desm[MLABERR[ilab + 1]][test]**2)

        if i==0:
            p0 = np.random.rand(3)
            p0[2] = np.abs(p0[2])  # Assicurati che s sia positivo
        else:
            p0=soln.x
        np.random.seed(42)
        nll = lambda *args: -CMRp_clipped(*args)
        initial = p0
        soln = minimize(nll, initial, args=(x, y, xerr, yerr),method="Powell")
        results[i, 0:3, ilab] = soln.x[0:3]
        results[i, 3, ilab] = np.median(x)

# Creazione delle funzioni di interpolazione per ogni parametro e ogni ilab
interp_funcs = {}
for idx, param in enumerate(['A', 'B', 's','med']):
    interp_funcs[param] = [interp1d(zrange, results[:, idx, k], kind='quadratic', fill_value="extrapolate") for k in range(3)]

def CMR(z):
    """ Restituisce i valori interpolati di A, B, s, e med per ogni ilab dato un z. """
    return [func(z) for sublist in interp_funcs.values() for func in sublist]



In [ ]:
CMR(0.46)

In [ ]:
|# Plot dei risultatia
iilab = 2
zz = np.linspace(0.05,1.5,1000)
plt.plot(zrange, results[:, 0,iilab], label='A')
plt.plot(zz, CMR(zz)[iilab], label='A-')
plt.plot(zrange, results[:, 1,iilab], label='B')
plt.plot(zz, CMR(zz)[iilab+3], label='B-')
plt.plot(zrange, results[:, 2,iilab], label='s')
plt.plot(zz, CMR(zz)[iilab+6], label='s-')
plt.legend()
plt.show()


In [ ]:
def model(z,ilab):
    vec = CMR(z)
    x = sptcat[MLAB[ilab+1]]
    y = sptcat[MLAB[ilab]] - sptcat[MLAB[ilab+1]]
    A = vec[ilab]
    B = vec[ilab+3]
    s = vec[ilab+6]
    med = vec[ilab+9]
    ymod = A + B * (x-med)
    return x,ymod,s

zlim = [0.,0.37,0.79,1.5]
def like_p_of_z(theta,sptcat):
    z,p,s0,s1,s2 = theta
    logl = np.zeros(3)
    vec = CMR(z)
    for ilab in [0,1,2]:
        x = sptcat[MLAB[ilab+1]]
        y = sptcat[MLAB[ilab]] - sptcat[MLAB[ilab+1]]
        xerr = sptcat[MLABERR[ilab+1]]
        yerr = np.sqrt(sptcat[MLABERR[ilab]]**2+sptcat[MLABERR[ilab+1]]**2)
        A = vec[ilab]
        B = vec[ilab+3]
#        if z<zlim[ilab+1]:
        s = vec[ilab+6]
#        else:
#            s = 100.
        med = vec[ilab+9]
        ymod = A + B * (x-med)
        ymoderr2 = yerr**2 + s**2 + (xerr * B)**2
        sigma = theta[2+ilab]
#        logl[ilab]= np.sum(np.log(p*norm.pdf(y, loc=ymod, scale=np.sqrt(ymoderr2))+(1-p)*(1/sigma)))
        logl[ilab]= np.sum(np.log(p*norm.pdf(y, loc=ymod, scale=np.sqrt(ymoderr2))+(1-p)*norm.pdf(y, loc=np.median(y), scale=sigma)))
    return np.sum(logl)

def prior_p_of_z(theta):
    z,p,s0,s1,s2 = theta
    if (z>0) & (z<1.2) & (p>0) & (p<1) & (s0>0) & (s1>0) & (s2>0):
        return 0
    else:
        return -np.inf

def post_p_of_z(theta,sptcat):
    lp = prior_p_of_z(theta)
    ll = like_p_of_z(theta,sptcat)
    if np.isfinite(lp+ll):
        return lp+ll
    else:
        return -np.inf

### Applying the model to SPT data

The interpolated relations are evaluated on SPT cluster galaxies and an MCMC sampler estimates the cluster properties.

In [ ]:
y6lim = [24.5,24,23.4,22.75]
from astropy.table import Table
sptcat = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/sptclust.0.60.fits")
sptcat = sptcat[(sptcat["FLAGS_FOOTPRINT"]==1) & (sptcat["FLAGS_GOLD"] == 0)& (sptcat["FLAGSTR"] == "ok") & (sptcat["FLAGS_FOREGROUND"] == 0) &
(sptcat["FITVD_FLAGS"] == 0) &  (sptcat["MASK_FLAGS"] == 0) & (np.abs(sptcat["SPREADERR_MODEL_G"]) <0.1) &
(sptcat["MAGERR_AUTO_G"] > 0) & (sptcat["MAGERR_AUTO_R"] > 0) & (sptcat["MAGERR_AUTO_I"] > 0) & (sptcat["MAGERR_AUTO_Z"] > 0)
& (sptcat["MAG_AUTO_G"] <y6lim[0]) & (sptcat["MAG_AUTO_R"] <y6lim[1]) & (sptcat["MAG_AUTO_I"] <y6lim[2]) & (sptcat["MAG_AUTO_Z"] <y6lim[3])]
#& (np.abs(sptcat["DEC"]-np.median(sptcat["DEC"]))<1./60) &
#(np.abs((sptcat["RA"]-np.median(sptcat["RA"]))*np.cos(np.deg2rad(np.median(sptcat["DEC"]))))<1./60)]
#sptcat

sptcat["MAG_G"] = sptcat["BDF_MAG_G_CORRECTED"]
sptcat["MAG_R"] = sptcat["BDF_MAG_R_CORRECTED"]
sptcat["MAG_I"] = sptcat["BDF_MAG_I_CORRECTED"]
sptcat["MAG_Z"] = sptcat["BDF_MAG_Z_CORRECTED"]

# sptcat["MAG_G"] = sptcat["MAG_AUTO_G"]
# sptcat["MAG_R"] = sptcat["MAG_AUTO_R"]
# sptcat["MAG_I"] = sptcat["MAG_AUTO_I"]
# sptcat["MAG_Z"] = sptcat["MAG_AUTO_Z"]

sptcat["MAGERR_G"] = sptcat["BDF_MAG_ERR_G"]
sptcat["MAGERR_R"] = sptcat["BDF_MAG_ERR_R"]
sptcat["MAGERR_I"] = sptcat["BDF_MAG_ERR_I"]
sptcat["MAGERR_Z"] = sptcat["BDF_MAG_ERR_Z"]
#MLAB = ["MAG_AUTO_G","MAG_AUTO_R","MAG_AUTO_I","MAG_AUTO_Z"]
#MLABERR = ["MAGERR_AUTO_G","MAGERR_AUTO_R","MAGERR_AUTO_I","MAGERR_AUTO_Z"]
#MLABERR = ["BDF_MAG_ERR_G","BDF_MAG_ERR_R","BDF_MAG_ERR_I","BDF_MAG_ERR_Z"]

print(post_p_of_z([0.25,0.9,1,1,1],sptcat),post_p_of_z([0.41,0.9,1,1,1],sptcat),post_p_of_z([0.93,0.9,1,1,1],sptcat))

In [ ]:
plt.plot(sptcat['RA'],sptcat['MAG_G'],'.')

In [ ]:
nll = lambda *args: -post_p_of_z(*args)
#soln = minimize(nll,[0.4,0.9,1,1,1], args=(sptcat),method="Powell")
#soln = minimize(nll,soln.x , args=(sptcat),method="Nelder-Mead")
#soln = minimize(nll,[0.5,0.1,0.4,0.175,0.15] , args=(sptcat),method="Nelder-Mead")
initial = [0.1,0.1,0.4,0.175,0.15]
bestf = np.inf
bestp = initial
soln = minimize(nll,initial , args=(sptcat))
for i in np.arange(0.1,1.3,0.1):
    initial[0] = i
    soln = minimize(nll,initial , args=(sptcat))
    if soln.fun < bestf:
        bestp = soln.x
        bestf = soln.fun
soln = minimize(nll,bestp , args=(sptcat))
print(soln, post_p_of_z(soln.x,sptcat))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
for ilab in range(3):
    x = sptcat[MLAB[ilab + 1]]
    y = sptcat[MLAB[ilab]] - sptcat[MLAB[ilab + 1]]
    xmod, ymod, s = model(soln.x[0], ilab)

    ax = axes[ilab]
    ax.scatter(x, y, alpha=0.5, label=f'ilab={ilab}')
    ax.plot(xmod, ymod, color='red', label='Model Fit')
    ax.set_title(f'Panel {ilab + 1} (ilab={ilab}) z={round(soln.x[0],2)}')
    ax.set_xlabel(MLAB[ilab + 1])
    ax.set_ylabel(f'{MLAB[ilab]} - {MLAB[ilab + 1]}')
    ax.legend()

plt.tight_layout()
plt.show()

print(soln.x[0])

In [ ]:
pos = soln.x * (1+1e-1 * np.random.randn(32, 5))
#pos = [0.2,0.9,1]
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, post_p_of_z, args=([sptcat]))
sampler.run_mcmc(pos, 500, progress=True);
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
flat_samples = sampler.get_chain(flat=True,discard=200)
plt.hist(flat_samples[:,0],bins=50)

In [ ]:
zrange = np.arange(0.05,1.5,0.075)
dz = zrange[1]-zrange[0]

#plt.legend()
for i,j in enumerate(zrange):
    test = (cosmosm["EZzphot"]<j+dz) & (cosmosm["EZzphot"]>=j) & (cosmosm["passive"])
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
    for ilab in range(3):
        x = desm[MLAB[ilab + 1]][test]
        y = desm[MLAB[ilab]][test] - desm[MLAB[ilab + 1]][test]
        xmod, ymod, s = model(np.mean(cosmosm["EZzphot"][test]), ilab)

        ax = axes[ilab]
        ax.scatter(x, y, alpha=0.5, label=f'ilab={ilab}')
        ax.plot(xmod, ymod, color='red', label='Model Fit')
        ax.set_title(f'Panel {ilab + 1} (ilab={ilab}) z='+str(np.mean(cosmosm["EZzphot"][test])))
        ax.set_xlabel(MLAB[ilab + 1])
        ax.set_ylabel(f'{MLAB[ilab]} - {MLAB[ilab + 1]}')
        ax.legend()

    plt.tight_layout()
    plt.show()



In [ ]:
filt = [(398,548),(568,716),(710,857),(850,1002)]
for i in filt:
    print(i[0]/400.-1,i[1]/400.-1)

In [ ]:
spt = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/SPTpol_500d_catalog_tablevOct3.fits")

In [ ]:
spt[spt["XI"]>10]

### Further analysis

The remaining cells load additional SPT catalogues and provide examples of exploratory plots and checks.